# YOLOv8n Training on Kaggle - Roboflow Dataset

Training YOLOv8n model using dataset from Roboflow Universe.

**Dataset**: [v5-j3ysc](https://universe.roboflow.com/quang-linh/v5-j3ysc/dataset/1)

## Setup Requirements
1. Enable GPU in Kaggle (Settings → Accelerator → GPU P100)
2. Get your Roboflow API key from [Roboflow Settings](https://app.roboflow.com/settings/api)
3. Run all cells sequentially

## 1. Environment Setup

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install required packages
!pip install -q ultralytics roboflow

print("✓ Packages installed successfully!")

In [ ]:
# Import libraries
import os
import yaml
import shutil
from pathlib import Path
from ultralytics import YOLO
from roboflow import Roboflow
import torch
import glob
from IPython.display import Image, display

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## 2. Download Dataset from Roboflow

### Get your API key:
1. Go to [Roboflow Settings](https://app.roboflow.com/settings/api)
2. Copy your Private API Key
3. Paste it in the cell below

In [ ]:
# ⚠️ IMPORTANT: Replace with your Roboflow API key
ROBOFLOW_API_KEY = "ZibH2Daoysfb8ESqVkWk"  # Get from https://app.roboflow.com/settings/api

# Dataset information
WORKSPACE = "quang-linh"
PROJECT = "v5-j3ysc"
VERSION = 1

print(f"Workspace: {WORKSPACE}")
print(f"Project: {PROJECT}")
print(f"Version: {VERSION}")

In [ ]:
# Initialize Roboflow and download dataset
print("Connecting to Roboflow...")
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

print(f"Downloading dataset: {WORKSPACE}/{PROJECT}...")
project = rf.workspace(WORKSPACE).project(PROJECT)
dataset = project.version(VERSION).download("yolov8")

DATASET_PATH = dataset.location
print(f"\n✓ Dataset downloaded to: {DATASET_PATH}")

## 3. Verify Dataset

In [ ]:
# Check dataset structure
print("Dataset structure:")
print("="*60)
!ls -la {DATASET_PATH}

print("\nDataset subdirectories:")
for item in os.listdir(DATASET_PATH):
    item_path = os.path.join(DATASET_PATH, item)
    if os.path.isdir(item_path):
        print(f"  📁 {item}/")
        for subitem in os.listdir(item_path)[:5]:  # Show first 5 items
            print(f"      - {subitem}")
    else:
        print(f"  📄 {item}")

In [ ]:
# Read dataset configuration
yaml_path = os.path.join(DATASET_PATH, 'data.yaml')

with open(yaml_path, 'r') as f:
    config = yaml.safe_load(f)

print("Dataset Configuration:")
print("="*60)
print(yaml.dump(config, default_flow_style=False, sort_keys=False))

# Extract info
num_classes = config.get('nc', len(config.get('names', [])))
class_names = config.get('names', [])

print(f"\nNumber of classes: {num_classes}")
print(f"Class names: {class_names}")

In [ ]:
# Count images in each split
def count_images(path):
    if not os.path.exists(path):
        return 0
    return len([f for f in os.listdir(path) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp'))])

train_path = os.path.join(DATASET_PATH, config.get('train', 'train/images'))
val_path = os.path.join(DATASET_PATH, config.get('val', 'valid/images'))
test_path = os.path.join(DATASET_PATH, config.get('test', 'test/images'))

train_count = count_images(train_path)
val_count = count_images(val_path)
test_count = count_images(test_path)

print("Dataset Statistics:")
print("="*60)
print(f"Train images: {train_count:,}")
print(f"Validation images: {val_count:,}")
print(f"Test images: {test_count:,}")
print(f"Total images: {train_count + val_count + test_count:,}")
print("="*60)

In [ ]:
# Display sample images
import random
import matplotlib.pyplot as plt
from PIL import Image as PILImage

# Get random sample images
sample_images = [f for f in os.listdir(train_path) if f.endswith(('.jpg', '.jpeg', '.png'))][:6]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Sample Training Images', fontsize=16, fontweight='bold')

for idx, img_name in enumerate(sample_images):
    img_path = os.path.join(train_path, img_name)
    img = PILImage.open(img_path)
    
    ax = axes[idx // 3, idx % 3]
    ax.imshow(img)
    ax.set_title(img_name, fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.show()

print(f"Image size example: {img.size}")

## 4. Training Configuration

In [ ]:
# Training hyperparameters
EPOCHS = 100  # Recommended: 100-300 for best results
IMG_SIZE = 640  # Standard YOLO size (can use 1280 for better accuracy)
BATCH_SIZE = 16  # Adjust based on GPU memory (P100: 16-32, T4: 8-16)
PATIENCE = 30  # Early stopping patience
PROJECT_NAME = 'yolov8n_training'
EXPERIMENT_NAME = 'roboflow_v5_j3ysc'

# Advanced settings
OPTIMIZER = 'AdamW'  # Options: 'SGD', 'Adam', 'AdamW'
LEARNING_RATE = 0.01  # Initial learning rate
WEIGHT_DECAY = 0.0005

print("Training Configuration:")
print("="*60)
print(f"Model: YOLOv8n")
print(f"Epochs: {EPOCHS}")
print(f"Image Size: {IMG_SIZE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Patience: {PATIENCE}")
print(f"Optimizer: {OPTIMIZER}")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Weight Decay: {WEIGHT_DECAY}")
print("="*60)

## 5. Initialize Model

In [ ]:
# Load YOLOv8n with pretrained COCO weights
model = YOLO('yolov8n.pt')

print("✓ YOLOv8n model loaded with pretrained weights")
print("\nModel Summary:")
model.info()

## 6. Train the Model

This will take approximately 2-4 hours depending on dataset size and epochs.

In [ ]:
# Start training
print("Starting training...")
print("="*60)

results = model.train(
    # Dataset
    data=yaml_path,
    
    # Training parameters
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    patience=PATIENCE,
    
    # Output
    project=PROJECT_NAME,
    name=EXPERIMENT_NAME,
    exist_ok=False,
    
    # Device
    device=0,  # GPU 0
    
    # Optimizer
    optimizer=OPTIMIZER,
    lr0=LEARNING_RATE,
    lrf=0.01,  # Final learning rate (lr0 * lrf)
    momentum=0.937,
    weight_decay=WEIGHT_DECAY,
    
    # Augmentation
    hsv_h=0.015,  # HSV-Hue augmentation
    hsv_s=0.7,  # HSV-Saturation augmentation
    hsv_v=0.4,  # HSV-Value augmentation
    degrees=0.0,  # Rotation (+/- deg)
    translate=0.1,  # Translation (+/- fraction)
    scale=0.5,  # Scaling (+/- gain)
    shear=0.0,  # Shear (+/- deg)
    perspective=0.0,  # Perspective (+/- fraction)
    flipud=0.0,  # Flip up-down probability
    fliplr=0.5,  # Flip left-right probability
    mosaic=1.0,  # Mosaic augmentation probability
    mixup=0.0,  # MixUp augmentation probability
    copy_paste=0.0,  # Copy-paste augmentation probability
    
    # Validation
    val=True,
    
    # Saving
    save=True,
    save_period=-1,  # Save checkpoint every x epochs (-1 = only save last and best)
    
    # Performance
    cache=False,  # Set True if enough RAM (faster training)
    workers=8,
    
    # Other
    pretrained=True,
    verbose=True,
    seed=0,
    deterministic=True,
    plots=True,
    
    # Advanced
    amp=True,  # Automatic Mixed Precision
    fraction=1.0,  # Dataset fraction to train on
)

print("\n" + "="*60)
print("✓ Training completed!")
print("="*60)

## 7. Evaluate Model Performance

In [ ]:
# Load best model
best_model_path = f'{PROJECT_NAME}/{EXPERIMENT_NAME}/weights/best.pt'
best_model = YOLO(best_model_path)

print(f"✓ Loaded best model from: {best_model_path}")
print(f"Model size: {os.path.getsize(best_model_path) / (1024 * 1024):.2f} MB")

In [ ]:
# Validate on validation set
print("Running validation...")
metrics = best_model.val(
    data=yaml_path,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    conf=0.25,  # Confidence threshold
    iou=0.6,  # IoU threshold for NMS
    device=0,
    plots=True,
    save_json=True,
)

print("\n" + "="*60)
print("VALIDATION METRICS")
print("="*60)
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP75: {metrics.box.map75:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")
print("="*60)

In [ ]:
# Per-class metrics
print("\nPER-CLASS METRICS (AP50):")
print("="*60)

for i in range(num_classes):
    class_name = class_names[i] if isinstance(class_names, list) else class_names.get(i, f'class_{i}')
    ap50 = metrics.box.ap50[i]
    ap = metrics.box.ap[i]
    
    print(f"{class_name:20s} | AP50: {ap50:.4f} | AP50-95: {ap:.4f}")

print("="*60)

## 8. Visualize Training Results

In [ ]:
# Training curves
results_path = f'{PROJECT_NAME}/{EXPERIMENT_NAME}'

results_img = f'{results_path}/results.png'
if os.path.exists(results_img):
    print("Training Curves:")
    display(Image(filename=results_img, width=1000))
else:
    print("Results plot not found")

In [ ]:
# Confusion Matrix
confusion_matrix = f'{results_path}/confusion_matrix.png'
if os.path.exists(confusion_matrix):
    print("Confusion Matrix:")
    display(Image(filename=confusion_matrix, width=800))
else:
    print("Confusion matrix not found")

In [ ]:
# PR Curve
pr_curve = f'{results_path}/PR_curve.png'
if os.path.exists(pr_curve):
    print("Precision-Recall Curve:")
    display(Image(filename=pr_curve, width=800))

# F1 Curve
f1_curve = f'{results_path}/F1_curve.png'
if os.path.exists(f1_curve):
    print("\nF1 Score Curve:")
    display(Image(filename=f1_curve, width=800))

In [ ]:
# Validation predictions
val_batches = sorted(glob.glob(f'{results_path}/val_batch*.jpg'))

if val_batches:
    print(f"Validation Predictions (showing {min(3, len(val_batches))} batches):")
    print("="*60)
    
    for batch_img in val_batches[:3]:
        print(f"\n{os.path.basename(batch_img)}")
        display(Image(filename=batch_img, width=900))
else:
    print("No validation batch images found")

## 9. Test on Sample Images

In [ ]:
# Get test images
if test_count > 0:
    test_source = test_path
    print(f"Using test set: {test_count} images")
else:
    test_source = val_path
    print(f"Test set not available, using validation set: {val_count} images")

# Get sample images
test_images = [os.path.join(test_source, f) for f in os.listdir(test_source) 
               if f.endswith(('.jpg', '.jpeg', '.png'))][:10]

print(f"\nRunning inference on {len(test_images)} sample images...")

In [ ]:
# Run predictions
predictions = best_model.predict(
    source=test_images,
    imgsz=IMG_SIZE,
    conf=0.25,  # Confidence threshold
    iou=0.45,  # IoU threshold for NMS
    device=0,
    save=True,
    save_txt=True,
    save_conf=True,
    project=f'{PROJECT_NAME}/predictions',
    name='test_samples',
    exist_ok=True,
)

print(f"✓ Predictions saved to: {PROJECT_NAME}/predictions/test_samples/")

In [ ]:
# Display predictions
pred_images = sorted(glob.glob(f'{PROJECT_NAME}/predictions/test_samples/*.jpg'))

print(f"\nSample Predictions ({len(pred_images)} images):")
print("="*60)

for pred_img in pred_images[:6]:  # Show first 6
    print(f"\n{os.path.basename(pred_img)}")
    display(Image(filename=pred_img, width=700))

In [ ]:
# Analyze predictions
print("\nPrediction Analysis:")
print("="*60)

total_detections = 0
class_counts = {name: 0 for name in class_names}

for result in predictions:
    boxes = result.boxes
    total_detections += len(boxes)
    
    for box in boxes:
        cls_id = int(box.cls[0])
        cls_name = class_names[cls_id] if isinstance(class_names, list) else class_names.get(cls_id, f'class_{cls_id}')
        class_counts[cls_name] = class_counts.get(cls_name, 0) + 1

print(f"Total detections: {total_detections}")
print(f"Average detections per image: {total_detections / len(predictions):.2f}")
print("\nDetections per class:")
for cls_name, count in class_counts.items():
    print(f"  {cls_name}: {count}")

## 10. Export Model

In [ ]:
# Export to ONNX format (recommended for deployment)
print("Exporting model to ONNX format...")
onnx_path = best_model.export(
    format='onnx',
    imgsz=IMG_SIZE,
    dynamic=False,  # Set True for dynamic input sizes
    simplify=True,  # Simplify ONNX model
)

print(f"✓ ONNX model exported to: {onnx_path}")
print(f"  Size: {os.path.getsize(onnx_path) / (1024 * 1024):.2f} MB")

In [ ]:
# Optional: Export to other formats
# Uncomment the format you need

# TorchScript
# torchscript_path = best_model.export(format='torchscript')
# print(f"TorchScript: {torchscript_path}")

# TensorFlow Lite
# tflite_path = best_model.export(format='tflite')
# print(f"TFLite: {tflite_path}")

# TensorFlow SavedModel
# saved_model_path = best_model.export(format='saved_model')
# print(f"TensorFlow SavedModel: {saved_model_path}")

# CoreML (for iOS)
# coreml_path = best_model.export(format='coreml')
# print(f"CoreML: {coreml_path}")

## 11. Model Information & Summary

In [ ]:
# Model architecture info
print("Model Architecture Summary:")
print("="*60)
best_model.info(verbose=True, detailed=True)

# Count parameters
total_params = sum(p.numel() for p in best_model.model.parameters())
trainable_params = sum(p.numel() for p in best_model.model.parameters() if p.requires_grad)

print(f"\nModel Statistics:")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Model size (PT): {os.path.getsize(best_model_path) / (1024 * 1024):.2f} MB")
print(f"  Model size (ONNX): {os.path.getsize(onnx_path) / (1024 * 1024):.2f} MB")

In [ ]:
# Create comprehensive training summary
summary_path = f'{PROJECT_NAME}/{EXPERIMENT_NAME}/TRAINING_SUMMARY.txt'

with open(summary_path, 'w', encoding='utf-8') as f:
    f.write("="*80 + "\n")
    f.write("YOLOv8n TRAINING SUMMARY\n")
    f.write("="*80 + "\n\n")
    
    f.write("DATASET INFORMATION\n")
    f.write("-" * 80 + "\n")
    f.write(f"Source: Roboflow Universe\n")
    f.write(f"Workspace: {WORKSPACE}\n")
    f.write(f"Project: {PROJECT}\n")
    f.write(f"Version: {VERSION}\n")
    f.write(f"URL: https://universe.roboflow.com/{WORKSPACE}/{PROJECT}/dataset/{VERSION}\n")
    f.write(f"\nDataset Statistics:\n")
    f.write(f"  Train images: {train_count:,}\n")
    f.write(f"  Validation images: {val_count:,}\n")
    f.write(f"  Test images: {test_count:,}\n")
    f.write(f"  Total images: {train_count + val_count + test_count:,}\n")
    f.write(f"  Number of classes: {num_classes}\n")
    f.write(f"  Class names: {class_names}\n")
    
    f.write("\n" + "="*80 + "\n")
    f.write("TRAINING CONFIGURATION\n")
    f.write("-" * 80 + "\n")
    f.write(f"Model: YOLOv8n\n")
    f.write(f"Epochs: {EPOCHS}\n")
    f.write(f"Image Size: {IMG_SIZE}\n")
    f.write(f"Batch Size: {BATCH_SIZE}\n")
    f.write(f"Patience: {PATIENCE}\n")
    f.write(f"Optimizer: {OPTIMIZER}\n")
    f.write(f"Learning Rate: {LEARNING_RATE}\n")
    f.write(f"Weight Decay: {WEIGHT_DECAY}\n")
    f.write(f"Device: {torch.cuda.get_device_name(0)}\n")
    
    f.write("\n" + "="*80 + "\n")
    f.write("VALIDATION METRICS\n")
    f.write("-" * 80 + "\n")
    f.write(f"mAP50-95: {metrics.box.map:.4f}\n")
    f.write(f"mAP50: {metrics.box.map50:.4f}\n")
    f.write(f"mAP75: {metrics.box.map75:.4f}\n")
    f.write(f"Precision: {metrics.box.mp:.4f}\n")
    f.write(f"Recall: {metrics.box.mr:.4f}\n")
    
    f.write("\nPer-Class Metrics (AP50):\n")
    for i in range(num_classes):
        cls_name = class_names[i] if isinstance(class_names, list) else class_names.get(i, f'class_{i}')
        f.write(f"  {cls_name:20s}: {metrics.box.ap50[i]:.4f}\n")
    
    f.write("\n" + "="*80 + "\n")
    f.write("MODEL INFORMATION\n")
    f.write("-" * 80 + "\n")
    f.write(f"Total parameters: {total_params:,}\n")
    f.write(f"Trainable parameters: {trainable_params:,}\n")
    f.write(f"Model size (PyTorch): {os.path.getsize(best_model_path) / (1024 * 1024):.2f} MB\n")
    f.write(f"Model size (ONNX): {os.path.getsize(onnx_path) / (1024 * 1024):.2f} MB\n")
    
    f.write("\n" + "="*80 + "\n")
    f.write("OUTPUT FILES\n")
    f.write("-" * 80 + "\n")
    f.write(f"Best model: {best_model_path}\n")
    f.write(f"Last model: {PROJECT_NAME}/{EXPERIMENT_NAME}/weights/last.pt\n")
    f.write(f"ONNX model: {onnx_path}\n")
    f.write(f"Results: {results_path}/results.csv\n")
    f.write(f"Plots: {results_path}/*.png\n")
    
    f.write("\n" + "="*80 + "\n")

print(f"✓ Training summary saved to: {summary_path}")

# Display summary
print("\n" + "="*80)
with open(summary_path, 'r', encoding='utf-8') as f:
    print(f.read())

## 12. Package Results for Download

In [ ]:
# List all important files
print("Important Files:")
print("="*80)

important_files = [
    (f'{PROJECT_NAME}/{EXPERIMENT_NAME}/weights/best.pt', 'Best model weights'),
    (f'{PROJECT_NAME}/{EXPERIMENT_NAME}/weights/last.pt', 'Last epoch weights'),
    (onnx_path, 'ONNX model'),
    (f'{PROJECT_NAME}/{EXPERIMENT_NAME}/results.csv', 'Training results CSV'),
    (f'{PROJECT_NAME}/{EXPERIMENT_NAME}/results.png', 'Training curves'),
    (f'{PROJECT_NAME}/{EXPERIMENT_NAME}/confusion_matrix.png', 'Confusion matrix'),
    (f'{PROJECT_NAME}/{EXPERIMENT_NAME}/PR_curve.png', 'PR curve'),
    (f'{PROJECT_NAME}/{EXPERIMENT_NAME}/F1_curve.png', 'F1 curve'),
    (summary_path, 'Training summary'),
]

total_size = 0
for file_path, description in important_files:
    if os.path.exists(file_path):
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        total_size += size_mb
        print(f"✓ {description:30s} | {size_mb:8.2f} MB | {file_path}")
    else:
        print(f"✗ {description:30s} | NOT FOUND | {file_path}")

print("="*80)
print(f"Total size: {total_size:.2f} MB")

In [ ]:
# Create ZIP archive with all results
import zipfile
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
zip_filename = f'yolov8n_roboflow_results_{timestamp}.zip'

print(f"Creating archive: {zip_filename}")
print("="*80)

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add weights
    print("Adding weights...")
    zipf.write(f'{PROJECT_NAME}/{EXPERIMENT_NAME}/weights/best.pt', 'weights/best.pt')
    zipf.write(f'{PROJECT_NAME}/{EXPERIMENT_NAME}/weights/last.pt', 'weights/last.pt')
    
    # Add ONNX model
    print("Adding ONNX model...")
    zipf.write(onnx_path, f'weights/{os.path.basename(onnx_path)}')
    
    # Add plots
    print("Adding plots...")
    for plot_file in glob.glob(f'{PROJECT_NAME}/{EXPERIMENT_NAME}/*.png'):
        zipf.write(plot_file, f'plots/{os.path.basename(plot_file)}')
    
    # Add CSV results
    print("Adding results...")
    csv_file = f'{PROJECT_NAME}/{EXPERIMENT_NAME}/results.csv'
    if os.path.exists(csv_file):
        zipf.write(csv_file, 'results.csv')
    
    # Add summary
    print("Adding summary...")
    zipf.write(summary_path, 'TRAINING_SUMMARY.txt')
    
    # Add sample predictions
    print("Adding sample predictions...")
    for pred_img in pred_images[:5]:
        zipf.write(pred_img, f'predictions/{os.path.basename(pred_img)}')
    
    # Add dataset config
    print("Adding dataset config...")
    zipf.write(yaml_path, 'data.yaml')

zip_size = os.path.getsize(zip_filename) / (1024 * 1024)
print("="*80)
print(f"✓ Archive created successfully!")
print(f"  Filename: {zip_filename}")
print(f"  Size: {zip_size:.2f} MB")
print(f"\n📥 Download this file from Kaggle's output panel (right sidebar)")

## 13. Quick Inference Function

In [ ]:
# Create a quick inference function for testing
def predict_image(image_path, conf=0.25, save=True):
    """
    Quick inference on a single image
    
    Args:
        image_path: Path to image file
        conf: Confidence threshold (0-1)
        save: Whether to save the result
    """
    results = best_model.predict(
        source=image_path,
        imgsz=IMG_SIZE,
        conf=conf,
        save=save,
        device=0,
    )
    
    # Display results
    for result in results:
        boxes = result.boxes
        print(f"\nDetections: {len(boxes)}")
        
        for box in boxes:
            cls_id = int(box.cls[0])
            conf_score = float(box.conf[0])
            cls_name = class_names[cls_id] if isinstance(class_names, list) else class_names.get(cls_id, f'class_{cls_id}')
            
            print(f"  {cls_name}: {conf_score:.2f}")
        
        # Show image
        result.show()
    
    return results

print("✓ Quick inference function created!")
print("\nUsage: predict_image('path/to/image.jpg', conf=0.25)")

In [ ]:
# Test the inference function
if test_images:
    print("Testing inference function on a sample image...")
    sample_result = predict_image(test_images[0], conf=0.25)

## 📊 Training Complete!

### What's Next?

1. **Download Results**: Get the ZIP file from Kaggle's output panel
2. **Test Locally**: Use the exported ONNX or PyTorch model
3. **Deploy**: Integrate into your application
4. **Fine-tune**: Adjust hyperparameters if needed

### Tips for Better Results

- **More epochs**: Try 150-300 epochs
- **Larger model**: Use YOLOv8s or YOLOv8m for better accuracy
- **Higher resolution**: Use `imgsz=1280` (slower but more accurate)
- **Data augmentation**: Adjust augmentation parameters
- **Learning rate**: Experiment with different values

### Common Issues

- **Out of Memory**: Reduce batch size or image size
- **Poor mAP**: Check dataset quality, increase epochs
- **Overfitting**: Increase augmentation, add more data
- **Slow training**: Enable caching (`cache=True`)

### Resources

- [Ultralytics Docs](https://docs.ultralytics.com/)
- [YOLOv8 Training Guide](https://docs.ultralytics.com/modes/train/)
- [Roboflow Universe](https://universe.roboflow.com/)
- [Model Export Guide](https://docs.ultralytics.com/modes/export/)

---

**Happy Training! 🚀**